## === Loading extension

- <span style="color:red"><b>TODO</b> - Hook for automatic loading if env.var INTELL_GENAI_ENABLE_JUPYTER_AI=true)</span>.

In [1]:
%load_ext intell_genai.stratio_ai_magics

In [23]:
%ai --help

Usage: %%ai [OPTIONS] MODEL_ID

  Invokes a language model identified by MODEL_ID, with the prompt being
  contained in all lines after the first. Both local model IDs and global
  model IDs (with the provider ID explicitly prefixed, followed by a colon)
  are accepted.

  To view available language models, please run `%ai list`.

Options:
  -f, --format [code|html|image|json|markdown|math|md|text]
                                  IPython display to use when rendering
                                  output. [default="markdown"]
  -n, --region-name TEXT          AWS region name, e.g. 'us-east-1'. Required
                                  for SageMaker provider; does nothing with
                                  other providers.
  -q, --request-schema TEXT       The JSON object the endpoint expects, with
                                  the prompt being substituted into any value
                                  that matches the string literal '<prompt>'.
                         

## === Listing available providers & custom commands/chains

- <b>New feature</b>: Whitelist is not available on jupyter-ai-magics ==> Implemented in stratio_ai_magics
- <b>New feature</b>: Custom commans & chains registration on magic plugin instantiation

In [2]:
%ai list

| Provider | Environment variable | Set? | Models |
|----------|----------------------|------|--------|
| `stratio_genai_provider` | Not applicable. | <abbr title="Not applicable">N/A</abbr> | Stratio GenAI Service: acts as a gateway to final LLM provider |

Aliases and custom commands:

| Name | Target |
|------|--------|
| `stratio_genai` | `stratio_genai_provider:*` |
| `joke` | *custom chain* |


## === Interacting with default provider: Stratio GenAI ~ Proxy to final provider

#### We don't need to specify MODEL_ID

In [10]:
%%ai

¿quien eres?

Soy un modelo de lenguaje desarrollado por OpenAI llamado GPT-3. Estoy diseñado para generar respuestas a preguntas y ayudar con diversas tareas.

<span style="color:red"><b>TODO</b> - specifing MODEL_ID other than stratio (~ enable openai in whitelist)</span>.

In [7]:
%%ai stratio_genai

¿quien eres?

Soy un modelo de lenguaje desarrollado por OpenAI llamado GPT-3. Estoy aquí para ayudarte con tus preguntas y conversar contigo. ¿En qué puedo ayudarte hoy?

#### Code generation

In [14]:
%%ai -f code

escribe una función que devuelva la suma de dos números en python

In [ ]:
def suma(num1, num2):
    return num1 + num2

In [15]:
%%ai stratio_genai -f code

escribe una función que devuelva el mínimo común multiplo de dos números

In [ ]:
def minimo_comun_multiplo(a, b):
    def mcm(a, b):
        m = max(a, b)
        while True:
            if m % a == 0 and m % b == 0:
                return m
            m += 1
    return mcm(a, b)

#### Formatted text generation

<span style="color:red"><b>Refinement</b> - Generated data is placed on cell output; ¿can it be placed on a new cell?</span>.

In [16]:
%%ai -f html

Escribe un Html con titulo y párrafos explicando los diferentes tipos de datos primarios de Python

In [18]:
%%ai -f markdown

Escribe un markdown con titulo y párrafos explicando los diferentes tipos de datos primarios de Python

# Tipos de datos primarios en Python

En Python, los tipos de datos primarios se utilizan para almacenar y representar diferentes tipos de valores. Estos tipos de datos primarios incluyen:

## 1. Números

Los números en Python se dividen en tres tipos principales: enteros (int), números de punto flotante (float) y números complejos (complex). Los enteros representan números enteros sin decimales, los números de punto flotante representan números con decimales y los números complejos representan números con una parte real e imaginaria.

Ejemplos de números en Python:

```python
entero = 10
punto_flotante = 3.14
complejo = 2 + 3j
```

## 2. Cadenas de texto

Las cadenas de texto (str) se utilizan para representar texto en Python. Pueden contener letras, números, símbolos y espacios. Las cadenas de texto se pueden definir utilizando comillas simples ('') o comillas dobles ("").

Ejemplos de cadenas de texto en Python:

```python
nombre = 'Juan'
frase = "Hola, ¿cómo estás?"
```

## 3. Booleanos

Los valores booleanos (bool) en Python representan la verdad o la falsedad. Solo pueden tener dos valores posibles: True (verdadero) o False (falso). Se utilizan principalmente en expresiones lógicas y condiciones.

Ejemplos de valores booleanos en Python:

```python
verdadero = True
falso = False
```

## 4. Listas

Las listas en Python son colecciones ordenadas y modificables de elementos. Pueden contener diferentes tipos de datos y se definen utilizando corchetes ([]). Los elementos de una lista se separan por comas.

Ejemplo de lista en Python:

```python
numeros = [1, 2, 3, 4, 5]
```

## 5. Tuplas

Las tuplas en Python son colecciones ordenadas e inmutables de elementos. Al igual que las listas, pueden contener diferentes tipos de datos, pero a diferencia de las listas, no se pueden modificar una vez creadas. Se definen utilizando paréntesis (()).

Ejemplo de tupla en Python:

```python
coordenadas = (10, 20)
```

## 6. Conjuntos

Los conjuntos en Python son colecciones no ordenadas de elementos únicos. No se pueden acceder a los elementos de un conjunto utilizando índices y no se mantienen en un orden específico. Se definen utilizando llaves ({}) o la función set().

Ejemplo de conjunto en Python:

```python
colores = {'rojo', 'verde', 'azul'}
```

## 7. Diccionarios

Los diccionarios en Python son colecciones de pares clave-valor. Cada elemento de un diccionario consiste en una clave y su valor correspondiente. Los diccionarios son modificables y se definen utilizando llaves ({}) y dos puntos (:).

Ejemplo de diccionario en Python:

```python
persona = {'nombre': 'Juan', 'edad': 30, 'ciudad': 'Madrid'}
```

Estos son los principales tipos de datos primarios en Python. Cada uno de ellos tiene sus propias características y se utilizan para diferentes propósitos en la programación.

In [26]:
%%ai -f math

Escribeme las ecuaciones de Maxwell en LaTeX

<IPython.core.display.Math object>

## === Interacting with Stratio GenAI deployed chains
- <b>POC</b>: Using JokeChain
- <span style="color:red"><b>TODO</b> - Integrate a more complex chain such as "SQL"

In [4]:
%%ai joke

jugadores de baloncesto enanos en Español

¿Qué hace un jugador de baloncesto enano cuando anota un triple?

¡Salta de alegría, pero no muy alto para no perderse la canasta!

## === Registering & executing custom chains in runtime

In [19]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# TODO ==> Custom LangChain BaseChatModel = StratioChatService(BaseChatModel) ~ client to Proxy chain 
from intell_genai.jupyter_ai.stratio_genai_provider.llm import StratioChatService

llm = StratioChatService(temperature=0.9)

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [20]:
%ai register companyname chain

Registered new alias `companyname`

In [21]:
%ai list

| Provider | Environment variable | Set? | Models |
|----------|----------------------|------|--------|
| `stratio_genai_provider` | Not applicable. | <abbr title="Not applicable">N/A</abbr> | Stratio GenAI Service: acts as a gateway to final LLM provider |

Aliases and custom commands:

| Name | Target |
|------|--------|
| `stratio_genai` | `stratio_genai_provider:*` |
| `joke` | *custom chain* |
| `companyname` | *custom chain* |


In [22]:
%%ai companyname

galletas saladas de pepinillo

/home/asoriano/miniconda3/envs/genai-jupyterlab/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"Saladillo Snacks" or "Pickled Crispies"